In [ ]:
from IPython.display import display
print("Bắt đầu thử nghiệm nhúng watermark và đánh giá chống tấn công...")

# Sử dụng ảnh đầu tiên làm demo
test_name, test_img = processed_images[0]
print(f"Sử dụng ảnh: {test_name}")

# Nhúng watermark sử dụng DWT
watermarked_img = embed_watermark_dwt(
    test_img, 
    watermark, 
    alpha=config.DWT_ALPHA,
    wavelet=config.DWT_WAVELET, 
    level=config.DWT_LEVEL
)

# Tính PSNR để đánh giá chất lượng
psnr_value = calculate_psnr(test_img, watermarked_img)

print(f"Nhúng watermark hoàn tất")
print(f"PSNR: {psnr_value:.2f} dB")

# Hàm hiển thị kết quả nhúng
def show_embedding_results(original, watermarked, psnr_val, watermark_img):
    """Hiển thị kết quả nhúng watermark"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Ảnh gốc
    axes[0,0].imshow(original)
    axes[0,0].set_title(f'Ảnh gốc\n{original.shape}')
    axes[0,0].axis('off')
    
    # Ảnh đã nhúng watermark
    axes[0,1].imshow(watermarked)
    axes[0,1].set_title(f'Ảnh đã nhúng watermark\nPSNR: {psnr_val:.2f} dB')
    axes[0,1].axis('off')
    
    # Watermark được nhúng
    axes[1,0].imshow(watermark_img, cmap='gray')
    axes[1,0].set_title(f'Watermark nhị phân\n{watermark_img.shape}')
    axes[1,0].axis('off')
    
    # Sự khác biệt (được phóng đại)
    diff_img = np.abs(original.astype(np.float32) - watermarked.astype(np.float32))
    diff_enhanced = np.clip(diff_img * 3, 0, 255).astype(np.uint8)
    axes[1,1].imshow(diff_enhanced)
    axes[1,1].set_title('Sự khác biệt (×3)')
    axes[1,1].axis('off')
    
    plt.suptitle('Kết quả nhúng watermark bằng DWT', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Hàm hiển thị kết quả tấn công và trích xuất
def show_attack_demo(original, watermarked, attacked, watermarks_extracted, attack_name, methods):
    """Hiển thị kết quả tấn công và trích xuất cho một loại tấn công cụ thể"""
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # Hàng 1: Quy trình tấn công
    axes[0,0].imshow(original)
    axes[0,0].set_title('Ảnh gốc')
    axes[0,0].axis('off')
    
    axes[0,1].imshow(watermarked)
    axes[0,1].set_title('Đã nhúng watermark')
    axes[0,1].axis('off')
    
    axes[0,2].imshow(attacked)
    axes[0,2].set_title(f'Sau tấn công\n({attack_name})')
    axes[0,2].axis('off')
    
    # Watermark gốc
    axes[0,3].imshow(watermark, cmap='gray')
    axes[0,3].set_title('Watermark gốc')
    axes[0,3].axis('off')
    
    # Hàng 2: Kết quả trích xuất từ 3 phương pháp
    for i, (method, wm_extracted) in enumerate(zip(methods, watermarks_extracted)):
        if i < 3:  # Chỉ hiển thị 3 phương pháp đầu tiên
            axes[1,i].imshow(wm_extracted, cmap='gray')
            ncc_val = calculate_ncc(watermark, wm_extracted)
            ber_val = calculate_ber(watermark, wm_extracted)
            axes[1,i].set_title(f'{method}\nNCC: {ncc_val:.3f}\nBER: {ber_val:.3f}')
            axes[1,i].axis('off')
    
    # Xóa subplot cuối cùng nếu chỉ có 3 phương pháp
    if len(methods) <= 3:
        axes[1,3].remove()
    
    plt.suptitle(f'Demo tấn công: {attack_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

show_embedding_results(test_img, watermarked_img, psnr_value, watermark)

# Thống kê về chất lượng nhúng
embedding_quality = {
    "Chỉ số": [
        "PSNR (dB)", 
        "Đánh giá chất lượng", 
        "Cường độ nhúng α", 
        "Wavelet", 
        "Level"
    ],
    "Giá trị": [
        f"{psnr_value:.2f}",
        "Chất lượng tốt (PSNR ≥ 30 dB)" if psnr_value >= 30 else (
            "Chất lượng chấp nhận được (20-30 dB)" if psnr_value >= 20 else "Chất lượng thấp (PSNR < 20 dB)"
        ),
        config.DWT_ALPHA,
        config.DWT_WAVELET,
        config.DWT_LEVEL
    ]
}

df_embedding_quality = pd.DataFrame(embedding_quality)
print("\nĐánh giá chất lượng nhúng:")
display(df_embedding_quality)

# Thử nghiệm tấn công và trích xuất
print(f"\nBắt đầu thử nghiệm tấn công hình học...")

# Khởi tạo danh sách lưu kết quả
attack_results = []
demo_attacks = ['rotate_-30', 'scale_0.5', 'crop_0.2']  # Chọn một vài tấn công để demo

# Thực hiện từng loại tấn công
for attack_name, attack_func in GEOMETRIC_ATTACKS.items():
    print(f"\nThử nghiệm tấn công: {attack_name}")
    
    # Áp dụng tấn công lên ảnh đã nhúng watermark
    attacked_img = attack_func(watermarked_img)
    
    # Trích xuất watermark bằng 3 phương pháp
    print("Trích xuất watermark...")
    
    # 1. Phương pháp đề xuất: DWT + SIFT
    wm_extracted_robust = extract_watermark_robust(
        attacked_img, test_img, config.WATERMARK_SIZE,
        alpha=config.DWT_ALPHA, wavelet=config.DWT_WAVELET, level=config.DWT_LEVEL
    )
    
    # 2. DWT đơn thuần (không SIFT)
    wm_extracted_dwt_only = extract_watermark_dwt_only(
        attacked_img, config.WATERMARK_SIZE,
        wavelet=config.DWT_WAVELET, level=config.DWT_LEVEL
    )
    
    # 3. SIFT đơn thuần (không DWT tham chiếu)
    wm_extracted_sift_only = extract_watermark_sift_only(
        attacked_img, test_img, config.WATERMARK_SIZE
    )
    
    # Tính toán metrics
    results = {
        'attack': attack_name,
        'ncc_robust': calculate_ncc(watermark, wm_extracted_robust),
        'ber_robust': calculate_ber(watermark, wm_extracted_robust),
        'ncc_dwt_only': calculate_ncc(watermark, wm_extracted_dwt_only), 
        'ber_dwt_only': calculate_ber(watermark, wm_extracted_dwt_only),
        'ncc_sift_only': calculate_ncc(watermark, wm_extracted_sift_only),
        'ber_sift_only': calculate_ber(watermark, wm_extracted_sift_only)
    }
    
    attack_results.append(results)
    
    # Hiển thị kết quả số
    result_table = pd.DataFrame({
        'Phương pháp': ['DWT+SIFT', 'DWT only', 'SIFT only'],
        'NCC': [results['ncc_robust'], results['ncc_dwt_only'], results['ncc_sift_only']],
        'BER': [results['ber_robust'], results['ber_dwt_only'], results['ber_sift_only']]
    })
    print("Kết quả:")
    display(result_table.round(3))
    
    # Hiển thị demo trực quan cho một số tấn công
    if attack_name in demo_attacks:
        print(f"   Hiển thị demo trực quan...")
        show_attack_demo(
            test_img, watermarked_img, attacked_img,
            [wm_extracted_robust, wm_extracted_dwt_only, wm_extracted_sift_only],
            attack_name,
            ['DWT+SIFT (đề xuất)', 'DWT đơn thuần', 'SIFT đơn thuần']
        )

# Tạo DataFrame để dễ xem
results_df = pd.DataFrame(attack_results)

print(f"\nTỔNG HỢP KẾT QUẢ THỬ NGHIỆM TẤN CÔNG")
print("="*60)
print(results_df.round(3).to_string(index=False))

# Tính trung bình hiệu suất
print(f"\nHIỆU SUẤT TRUNG BÌNH:")
print(f"  DWT+SIFT (đề xuất): NCC={results_df['ncc_robust'].mean():.3f}, BER={results_df['ber_robust'].mean():.3f}")
print(f"  DWT only:           NCC={results_df['ncc_dwt_only'].mean():.3f}, BER={results_df['ber_dwt_only'].mean():.3f}")
print(f"  SIFT only:          NCC={results_df['ncc_sift_only'].mean():.3f}, BER={results_df['ber_sift_only'].mean():.3f}")
